In [1]:
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

%matplotlib inline

In [2]:
#export
from exp.datasets import *

In [3]:
#export

from lxml import etree


In [4]:
#export

import ciso8601
import pandas as pd

In [5]:
path=untar_data(URLs.BPIC_2017)

BPIC_2017/BPI_Challenge_2017.xes.gz


In [6]:
path=untar_data(URLs.BPIC_2012)

BPIC_2012/BPI_Challenge_2012.xes.gz


In [7]:
#export

class XES_Constants():
    
    # XES TAGS
    TAG_BOOLEAN = 'boolean'
    TAG_DATE = 'date'
    TAG_EVENT = 'event'
    TAG_FLOAT = 'float'
    TAG_ID = 'id'
    TAG_INT = 'int'
    TAG_LOG = 'log'
    TAG_STRING = 'string'
    TAG_TRACE = 'trace'
    TAG_VALUES = 'values'
    
    # Not sure what these are doing!
    TAG_EXTENSION = 'extension'
    TAG_LIST = 'list'
    TAG_GLOBAL = 'global'
    TAG_CLASSIFIER = 'classifier'

    
    # XES/INTERNAL KEYS
    KEY_CHILDREN = 'children'
    KEY_KEY = 'key'
    KEY_KEYS = 'keys'
    KEY_NAME = 'name'
    KEY_PREFIX = 'prefix'
    KEY_SCOPE = 'scope'
    KEY_URI = 'uri'
    KEY_VALUE = 'value'

    DEFAULT_NAME_KEY = 'concept:name'
    DEFAULT_TIMESTAMP_KEY = 'time:timestamp'
    DEFAULT_START_TIMESTAMP_KEY = 'start_timestamp'
    DEFAULT_TRACEID_KEY = 'concept:name'
    DEFAULT_RESOURCE_KEY = 'org:resource'
    DEFAULT_TRANSITION_KEY = "lifecycle:transition"


In [38]:
#export

from collections.abc import Mapping, Sequence

class EventLog():
    def __init__(self,traceAttributeColumns=None,eventColumns=None): 
        self._extensions = {}
        self._classifiers = {}
        self._attributes = {}

        self._schema = {}
        self._eventList=[]
        self._eventColumns=eventColumns
        self._traceAttributeColumns=traceAttributeColumns
        self._traceAttributeList=[]


    def __len__(self): return len(self._traceAttributeList)
    
    def append(self,trace):
        if not self._eventColumns: self._eventColumns=['trace_id','event_id',*trace.events[0].keys()]
        trace_id=trace.attributes[XES_Constants.DEFAULT_TRACEID_KEY]
        for i,event in enumerate(trace.events):
            ll=[]
            for a in self._eventColumns[2:]:
                #print(event)
                ll.append(event.get(a,None))
            self._eventList.append([trace_id,i,*ll])
        
        if not self._traceAttributeColumns: self._traceAttributeColumns=trace.attributes.keys()
        ll=[]
        for a in self._traceAttributeColumns:
            #print(event)
            ll.append(trace.attributes.get(a,None))
        self._traceAttributeList.append(ll)
               
    def _get_classifiers(self): return self._classifiers

    def _get_extensions(self): return self._extensions
    def _get_attributes(self): return self._attributes

    def _trace_attributes(self): 
        return pd.DataFrame(self._traceAttributeList,columns=self._traceAttributeColumns)
    def _event_stream(self,convertTime=True):
        df=pd.DataFrame(self._eventList,columns=self._eventColumns)
        if convertTime: 
            df[XES_Constants.DEFAULT_TIMESTAMP_KEY] = pd.to_datetime(df[XES_Constants.DEFAULT_TIMESTAMP_KEY],utc=True)
        return df
    
    def _get_schema(self): return self._schema

    events = property(_event_stream)
    traceAttributes = property(_trace_attributes)
    
    extensions = property(_get_extensions)
    attributes = property(_get_attributes)
    classifiers = property(_get_classifiers)
    schema = property(_get_schema)


    
class Trace():
    def __init__(self,case_tag=XES_Constants.DEFAULT_NAME_KEY): 
        self.events=[]
        self._attributes = {}
        self._case_tag=case_tag

    
    def __len__(self): return len(self.events)
    
    def _get_attributes(self): return self._attributes
    
    def __repr__(self): 
        return f"Trace {self._attributes[self._case_tag]}"
    
    def __str__(self): 
        return f"Trace {self._attributes[self._case_tag]}"
    
    def append(self,event): self.events.append(event)
    
    attributes = property(_get_attributes)

    
class Event(Mapping):
    def __init__(self, *args, **kw):
        self._dict = dict(*args, **kw)

    def __getitem__(self, key):
        return self._dict[key]

    def __setitem__(self, key, value):
        self._dict[key] = value

    def __iter__(self):
        return iter(self._dict)

    def __len__(self):
        return len(self._dict)

    def __delitem__(self, key):
        del self._dict[key]

    def __repr__(self):
        return str(dict(self))
    
    def __str__(self):
        return str(dict(self))

    
    


In [44]:
#export

def __parse_attribute(elem, store, key, value, tree):
#    print(elem, store, key, value, tree)
    if len(elem.getchildren()) == 0:
        store[key] = value
    else:
        store[key] = {XES_Constants.KEY_VALUE: value, XES_Constants.KEY_CHILDREN: {}}
        if elem.getchildren()[0].tag.endswith(XES_Constants.TAG_VALUES):
            tree[elem] = store[key][XES_Constants.KEY_CHILDREN]
            tree[elem.getchildren()[0]] = tree[elem]
        else:
            tree[elem] = store[key][XES_Constants.KEY_CHILDREN]
    return tree

EVENT_END = 'end'
EVENT_START = 'start'



def import_xes(path,extensions=True,classifiers=True,schema=True,log_attributes=True,max_number_traces=None):
    filename = str(path)    


    context = etree.iterparse(filename, events=['start', 'end'])




    log = None
    trace= None
    event = None


    tree = {}



    for tree_event, elem in context:

        if tree_event == EVENT_START:  # starting to read
            parent = tree[elem.getparent()] if elem.getparent() in tree else None

            if elem.tag.endswith(XES_Constants.TAG_STRING):
                if parent is not None:
                    tree = __parse_attribute(elem, parent, elem.get(XES_Constants.KEY_KEY),
                                             elem.get(XES_Constants.KEY_VALUE), tree)
                continue

            elif elem.tag.endswith(XES_Constants.TAG_DATE):
                try:
                    dt = ciso8601.parse_datetime(elem.get(XES_Constants.KEY_VALUE))
                    tree = __parse_attribute(elem, parent, elem.get(XES_Constants.KEY_KEY), dt, tree)

                except TypeError:
                    print("failed to parse date: " + str(elem.get(XES_Constants.KEY_VALUE)))
                    # if counter ==max_runs: break
                    #counter +=1
                except ValueError:
                    print("failed to parse date: " + str(elem.get(XES_Constants.KEY_VALUE)))
                continue

            elif elem.tag.endswith(XES_Constants.TAG_EVENT):
                if event is not None:
                    raise SyntaxError('file contains <event> in another <event> tag')
                event=Event()
                tree[elem]=event
                continue

            elif elem.tag.endswith(XES_Constants.TAG_TRACE):
                if max_number_traces != None and len(log) >=max_number_traces : break
                if trace is not None: raise SyntaxError('file contains nested traces')
                trace=Trace()
                tree[elem] = trace.attributes

                continue

            elif elem.tag.endswith(XES_Constants.TAG_FLOAT):
                if parent is not None:
                    try:
                        val = float(elem.get(XES_Constants.KEY_VALUE))
                        tree = __parse_attribute(elem, parent, elem.get(XES_Constants.KEY_KEY), val, tree)
                    except ValueError:
                        logging.info("failed to parse float: " + str(elem.get(XES_Constants.KEY_VALUE)))
                continue

            elif elem.tag.endswith(XES_Constants.TAG_INT):
                if parent is not None:
                    try:
                        val = int(elem.get(XES_Constants.KEY_VALUE))
                        tree = __parse_attribute(elem, parent, elem.get(XES_Constants.KEY_KEY), val, tree)
                    except ValueError:
                        logging.info("failed to parse int: " + str(elem.get(XES_Constants.KEY_VALUE)))
                continue

            elif elem.tag.endswith(XES_Constants.TAG_BOOLEAN):
                if parent is not None:
                    try:
                        val0 = elem.get(XES_Constants.KEY_VALUE)
                        val = False
                        if str(val0).lower() == "true":
                            val = True
                        tree = __parse_attribute(elem, parent, elem.get(XES_Constants.KEY_KEY), val, tree)
                    except ValueError:
                        logging.info("failed to parse boolean: " + str(elem.get(XES_Constants.KEY_VALUE)))
                continue


            elif elem.tag.endswith(XES_Constants.TAG_LIST):
                if parent is not None:
                    # lists have no value, hence we put None as a value
                    tree = __parse_attribute(elem, parent, elem.get(XES_Constants.KEY_KEY), None, tree)
                continue


            elif elem.tag.endswith(XES_Constants.TAG_ID):
                if parent is not None:
                    tree = __parse_attribute(elem, parent, elem.get(XES_Constants.KEY_KEY),
                                             elem.get(XES_Constants.KEY_VALUE), tree)
                continue

            elif elem.tag.endswith(XES_Constants.TAG_EXTENSION) and extensions:
                if log is None:
                    raise SyntaxError('extension found outside of <log> tag')
                if elem.get(XES_Constants.KEY_NAME) is not None and elem.get(
                        XES_Constants.KEY_PREFIX) is not None and elem.get(XES_Constants.KEY_URI) is not None:
                    log.extensions[elem.get(XES_Constants.KEY_NAME)] = {
                        XES_Constants.KEY_PREFIX: elem.get(XES_Constants.KEY_PREFIX),
                        XES_Constants.KEY_URI: elem.get(XES_Constants.KEY_URI)}
                continue

            elif elem.tag.endswith(XES_Constants.TAG_GLOBAL) and schema:
                if log is None:
                    raise SyntaxError('global found outside of <log> tag')
                if elem.get(XES_Constants.KEY_SCOPE) is not None:
                    log._schema[elem.get(XES_Constants.KEY_SCOPE)] = {}
                    tree[elem] = log._schema[elem.get(XES_Constants.KEY_SCOPE)]
                continue

            elif elem.tag.endswith(XES_Constants.TAG_CLASSIFIER) and classifiers:
                if log is None:
                    raise SyntaxError('classifier found outside of <log> tag')
                if elem.get(XES_Constants.KEY_KEYS) is not None:
                    classifier_value = elem.get(XES_Constants.KEY_KEYS)
                    if "'" in classifier_value:
                        log.classifiers[elem.get(XES_Constants.KEY_NAME)] = [x for x in classifier_value.split("'")
                                                                                if x.strip()]
                    else:
                        log.classifiers[elem.get(XES_Constants.KEY_NAME)] = classifier_value.split()
                continue


            elif elem.tag.endswith(XES_Constants.TAG_LOG):
                log=EventLog()
                if log_attributes: tree[elem]=log.attributes 
                continue

        elif tree_event == EVENT_END:

            if elem.tag.endswith(XES_Constants.TAG_EVENT):
                if trace is not None:
                    trace.append(event)
                    event=None
                continue

            elif elem.tag.endswith(XES_Constants.TAG_TRACE):
                if log is not None:
                    log.append(trace)
                    trace=None
                continue

            elif elem.tag.endswith(XES_Constants.TAG_LOG):
                continue
    context=0
    del context
    tree=0
    del tree
    return log




In [40]:
! /home/lahann/anaconda3/envs/fastpm/bin/python notebook2script.py 03_fastpm_eventlog.ipynb

Converted 03_fastpm_eventlog.ipynb to exp/eventlog.py


In [28]:
log.extensions


{'Metadata_Time': {'prefix': 'meta_time',
  'uri': 'http://www.xes-standard.org/meta_time.xesext'},
 'Lifecycle': {'prefix': 'lifecycle',
  'uri': 'http://www.xes-standard.org/lifecycle.xesext'},
 'Metadata_Lifecycle': {'prefix': 'meta_life',
  'uri': 'http://www.xes-standard.org/meta_life.xesext'},
 'Organizational': {'prefix': 'org',
  'uri': 'http://www.xes-standard.org/org.xesext'},
 'Metadata_Organizational': {'prefix': 'meta_org',
  'uri': 'http://www.xes-standard.org/meta_org.xesext'},
 'Time': {'prefix': 'time', 'uri': 'http://www.xes-standard.org/time.xesext'},
 'Metadata_Concept': {'prefix': 'meta_concept',
  'uri': 'http://www.xes-standard.org/meta_concept.xesext'},
 '3TU metadata': {'prefix': 'meta_3TU',
  'uri': 'http://www.xes-standard.org/meta_3TU.xesext'},
 'Concept': {'prefix': 'concept',
  'uri': 'http://www.xes-standard.org/concept.xesext'},
 'General metadata': {'prefix': 'meta_general',
  'uri': 'http://www.xes-standard.org/meta_general.xesext'},
 'Semantic': {'pre

In [10]:
%%time 
%%memit
x=0

peak memory: 117.02 MiB, increment: 0.00 MiB
CPU times: user 71.2 ms, sys: 29.1 ms, total: 100 ms
Wall time: 241 ms


In [45]:
%%time 
%%memit
log=import_xes(path)


peak memory: 1733.21 MiB, increment: 90.53 MiB
CPU times: user 8.42 s, sys: 822 ms, total: 9.24 s
Wall time: 9.44 s


In [22]:
%%memit

df=log.events


peak memory: 1641.68 MiB, increment: 0.00 MiB


In [23]:
df

,trace_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,112,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00
1,173688,112,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00
2,173688,112,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00
3,173688,112,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00
4,173688,None,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00
5,173688,10862,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00
6,173688,10862,COMPLETE,O_SELECTED,2011-10-01 11:45:09.243000+02:00
7,173688,10862,COMPLETE,A_FINALIZED,2011-10-01 11:45:09.243000+02:00
8,173688,10862,COMPLETE,O_CREATED,2011-10-01 11:45:11.197000+02:00
9,173688,10862,COMPLETE,O_SENT,2011-10-01 11:45:11.380000+02:00


In [24]:
df[DEFAULT_TIMESTAMP_KEY] = pd.to_datetime(df[DEFAULT_TIMESTAMP_KEY],utc=True)
df

,trace_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00
1,173688,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00
2,173688,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00
3,173688,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00
4,173688,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00
5,173688,10862,COMPLETE,A_ACCEPTED,2011-10-01 09:42:43.308000+00:00
6,173688,10862,COMPLETE,O_SELECTED,2011-10-01 09:45:09.243000+00:00
7,173688,10862,COMPLETE,A_FINALIZED,2011-10-01 09:45:09.243000+00:00
8,173688,10862,COMPLETE,O_CREATED,2011-10-01 09:45:11.197000+00:00
9,173688,10862,COMPLETE,O_SENT,2011-10-01 09:45:11.380000+00:00


In [ ]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_')])

In [ ]:
log.events

In [ ]:
log.events

In [ ]:
log.traces[0].events

In [ ]:
import pandas as pd

In [30]:
log.classifiers

{'Activity classifier': ['concept:name', 'lifecycle:transition'],
 'Resource classifier': ['org:resource']}

In [ ]:
%%time
%%memit
df_new=pd.DataFrame()
for i in progress_bar(range(len(log.traces))):
    df=pd.DataFrame(log.traces[i].events)
    df['trace_id']=log.traces[i].attributes[XES_Constants.DEFAULT_TRACEID_KEY]
    df_new=pd.concat([df_new, df], ignore_index=True)
df_new